In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torchvision import models, transforms
import numpy as np

In [2]:
train_dir = os.path.join('dataset', 'part_one_dataset', 'train_data')
eval_dir = os.path.join('dataset', 'part_one_dataset', 'eval_data')
train_path = os.path.join(train_dir, '1_train_data.tar.pth')
eval_path = os.path.join(eval_dir, '1_eval_data.tar.pth')

t = torch.load(train_path, weights_only = False)

In [3]:
import numpy as np
from sklearn.metrics.pairwise import cosine_distances, manhattan_distances

class LWP:
    """Learning Vector Prototypes with configurable distance function"""
    
    DISTANCE_FUNCTIONS = {
        'euclidean': lambda x, y: np.linalg.norm(x - y),
        'cosine': lambda x, y: cosine_distances(x.reshape(1, -1), y.reshape(1, -1))[0][0],
        'manhattan': lambda x, y: manhattan_distances(x.reshape(1, -1), y.reshape(1, -1))[0][0],
        'minkowski': lambda x, y, p=2: np.power(np.sum(np.power(np.abs(x - y), p)), 1/p)
    }
    
    def __init__(self, distance_metric='euclidean', **distance_params):
        """
            distance_params (dict): Additional parameters for the distance function
        """
        self.prototypes = {}
        self.class_counts = {i: 0 for i in range(10)}
        
        if callable(distance_metric):
            self.distance_fn = distance_metric
        elif distance_metric in self.DISTANCE_FUNCTIONS:
            if distance_metric == 'minkowski':
                p = distance_params.get('p', 2)
                self.distance_fn = lambda x, y: self.DISTANCE_FUNCTIONS[distance_metric](x, y, p)
            else:
                self.distance_fn = self.DISTANCE_FUNCTIONS[distance_metric]
        else:
            raise ValueError(f"Unknown distance metric: {distance_metric}. " 
                           f"Available metrics: {list(self.DISTANCE_FUNCTIONS.keys())}")

    def fit(self, features, labels):
        unique_labels = np.unique(labels)
        for label in unique_labels:
            samples = features[labels == label]
            num_samples = len(samples)
            
            if label not in self.prototypes:  # Original condition was: if label not in self.prototypes
                self.prototypes[label] = samples.mean(axis=0)
                self.class_counts[label] = len(samples)
            else:
                self.class_counts[label] += len(samples)
                self.prototypes[label] = (
                    (self.class_counts[label] - num_samples) / self.class_counts[label] * self.prototypes[label] +
                    num_samples / self.class_counts[label] * samples.mean(axis=0)
                )
    
            
    

    def predict(self, features):
        preds = []
        for feature in features:
            distances = {
                label: self.distance_fn(feature, proto)
                for label, proto in self.prototypes.items()
            }
            preds.append(min(distances, key=distances.get))
        return np.array(preds)

In [4]:
print (max(t['targets']))

9


In [5]:
print ( t['data'].shape)

(2500, 32, 32, 3)


In [6]:
data, targets = t['data'], t['targets'] 
data=data.reshape(data.shape[0], -1)
print(data.shape)

(2500, 3072)


In [7]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.preprocessing import normalize

In [8]:
data1, targets1= t['data'], t['targets']
data1=data1.reshape(data1.shape[0], -1)
data1=normalize(data1)
dataloader=DataLoader(data1, batch_size=32, shuffle=False)
lwp_model = LWP(distance_metric='cosine')  # LWP model with cosine distance
lwp_model.fit(data1, targets1) 

In [9]:

print("Prototypes after fit:", lwp_model.prototypes)


Prototypes after fit: {0: array([0.01729068, 0.01912501, 0.02102479, ..., 0.01516719, 0.0159201 ,
       0.01601679]), 1: array([0.01837207, 0.01810205, 0.01747006, ..., 0.01770059, 0.01728502,
       0.01624851]), 2: array([0.0164336 , 0.01742426, 0.01454287, ..., 0.01747445, 0.01748331,
       0.01434357]), 3: array([0.01795165, 0.0166975 , 0.01529295, ..., 0.01780526, 0.01648547,
       0.01536131]), 4: array([0.01489708, 0.01576391, 0.01352558, ..., 0.0190007 , 0.01918124,
       0.01478338]), 5: array([0.01479913, 0.01467359, 0.01309324, ..., 0.01706052, 0.01606419,
       0.01416187]), 6: array([0.0163735 , 0.01605917, 0.01306188, ..., 0.01921913, 0.01844561,
       0.01541324]), 7: array([0.01802458, 0.01884868, 0.01836053, ..., 0.01945586, 0.01867545,
       0.01490231]), 8: array([0.01797723, 0.02032782, 0.02269468, ..., 0.01198278, 0.01380034,
       0.01474392]), 9: array([0.02226604, 0.02317505, 0.02398837, ..., 0.01793398, 0.01759059,
       0.0167393 ])}


In [10]:
# Test LWP fit
print("Testing LWP fit...")
lwp_model = LWP(distance_metric='cosine')
lwp_model.fit(data1, targets1)

# Check prototypes
print("Prototypes initialized:")
for label, prototype in lwp_model.prototypes.items():
    print(f"Class {label}: Prototype shape = {prototype.shape}")


Testing LWP fit...
Prototypes initialized:
Class 0: Prototype shape = (3072,)
Class 1: Prototype shape = (3072,)
Class 2: Prototype shape = (3072,)
Class 3: Prototype shape = (3072,)
Class 4: Prototype shape = (3072,)
Class 5: Prototype shape = (3072,)
Class 6: Prototype shape = (3072,)
Class 7: Prototype shape = (3072,)
Class 8: Prototype shape = (3072,)
Class 9: Prototype shape = (3072,)


In [11]:
print("Features shape:", data1.shape)
print("Labels shape:", targets1.shape)
print("Unique labels:", np.unique(targets1))


Features shape: (2500, 3072)
Labels shape: (2500,)
Unique labels: [0 1 2 3 4 5 6 7 8 9]


In [12]:
import os
import numpy as np
import torch
import torch.nn.functional as F
from sklearn.preprocessing import normalize
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics.pairwise import cosine_distances

In [13]:
print("Prototypes:", lwp_model.prototypes)

Prototypes: {0: array([0.01729068, 0.01912501, 0.02102479, ..., 0.01516719, 0.0159201 ,
       0.01601679]), 1: array([0.01837207, 0.01810205, 0.01747006, ..., 0.01770059, 0.01728502,
       0.01624851]), 2: array([0.0164336 , 0.01742426, 0.01454287, ..., 0.01747445, 0.01748331,
       0.01434357]), 3: array([0.01795165, 0.0166975 , 0.01529295, ..., 0.01780526, 0.01648547,
       0.01536131]), 4: array([0.01489708, 0.01576391, 0.01352558, ..., 0.0190007 , 0.01918124,
       0.01478338]), 5: array([0.01479913, 0.01467359, 0.01309324, ..., 0.01706052, 0.01606419,
       0.01416187]), 6: array([0.0163735 , 0.01605917, 0.01306188, ..., 0.01921913, 0.01844561,
       0.01541324]), 7: array([0.01802458, 0.01884868, 0.01836053, ..., 0.01945586, 0.01867545,
       0.01490231]), 8: array([0.01797723, 0.02032782, 0.02269468, ..., 0.01198278, 0.01380034,
       0.01474392]), 9: array([0.02226604, 0.02317505, 0.02398837, ..., 0.01793398, 0.01759059,
       0.0167393 ])}


In [14]:
prototypes= lwp_model.prototypes
print(prototypes)

{0: array([0.01729068, 0.01912501, 0.02102479, ..., 0.01516719, 0.0159201 ,
       0.01601679]), 1: array([0.01837207, 0.01810205, 0.01747006, ..., 0.01770059, 0.01728502,
       0.01624851]), 2: array([0.0164336 , 0.01742426, 0.01454287, ..., 0.01747445, 0.01748331,
       0.01434357]), 3: array([0.01795165, 0.0166975 , 0.01529295, ..., 0.01780526, 0.01648547,
       0.01536131]), 4: array([0.01489708, 0.01576391, 0.01352558, ..., 0.0190007 , 0.01918124,
       0.01478338]), 5: array([0.01479913, 0.01467359, 0.01309324, ..., 0.01706052, 0.01606419,
       0.01416187]), 6: array([0.0163735 , 0.01605917, 0.01306188, ..., 0.01921913, 0.01844561,
       0.01541324]), 7: array([0.01802458, 0.01884868, 0.01836053, ..., 0.01945586, 0.01867545,
       0.01490231]), 8: array([0.01797723, 0.02032782, 0.02269468, ..., 0.01198278, 0.01380034,
       0.01474392]), 9: array([0.02226604, 0.02317505, 0.02398837, ..., 0.01793398, 0.01759059,
       0.0167393 ])}


In [38]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import cosine_similarity
# Function to compute cosine similarity and select top samples
def select_top_samples(embeddings, centroids, top_k=50):
    """
    Select top-k% samples with highest cosine similarity to centroids.
    """
    similarities = cosine_similarity(embeddings, centroids)
    pseudo_labels = np.argmax(similarities, axis=1)
    max_similarities = np.max(similarities, axis=1)

    # Sort by similarity and select top k% samples
    sorted_indices = np.argsort(max_similarities)[::-1]
    top_count = len(sorted_indices) * top_k // 100
    top_indices = sorted_indices[:top_count]

    return embeddings[top_indices], pseudo_labels[top_indices]

In [39]:
def process_dataset_with_knn(embed_dir, dataset_idx, lwp_model, k=5, top_k=50):
    print(f"Processing dataset {dataset_idx} with kNN...")

    # Load embeddings
    embed_path = os.path.join(embed_dir, f'train_embeds_{dataset_idx}.pt')
    embeddings = torch.load(embed_path)

    # Compute pseudo-labels and select top samples
    centroids = np.vstack([proto for _, proto in sorted(lwp_model.prototypes.items())])
    top_embeddings, top_pseudo_labels = select_top_samples(embeddings, centroids, top_k=top_k)

    # Train kNN on the top samples
    knn = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    knn.fit(top_embeddings, top_pseudo_labels)

    # Use kNN to assign pseudo-labels for all embeddings
    all_pseudo_labels = knn.predict(embeddings)
    print(f"Dataset {dataset_idx}: Assigned pseudo-labels using kNN")

    # Update prototypes (class centroids) with kNN pseudo-labeled samples
    for label in np.unique(all_pseudo_labels):
        class_embeddings = embeddings[all_pseudo_labels == label]
        if class_embeddings.size > 0:
            centroid = class_embeddings.mean(axis=0)
            lwp_model.prototypes[label] = centroid

    print(f"Dataset {dataset_idx}: Updated prototypes for classes {list(lwp_model.prototypes.keys())}")

In [33]:
# Directories for embeddings
part_one_embed_dir = 'part_1_vit_embeds'
part_two_embed_dir = 'part_2_vit_embeds'

# Load D1 embeddings and targets
train_embed_path = os.path.join(part_one_embed_dir, 'train_embeds_1.pt')
eval_embed_path = os.path.join(part_one_embed_dir, 'eval_embeds_1.pt')
train_embeddings = torch.load(train_embed_path)
eval_embeddings = torch.load(eval_embed_path)

C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\2342488638.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_embeddings = torch.load(train_embed_path)
C:\U

In [34]:
train_path = os.path.join('dataset', 'part_one_dataset', 'train_data', '1_train_data.tar.pth')
data = torch.load(train_path)
targets = data['targets']

# Initialize and fit LWP model
lwp_model = LWP(distance_metric='cosine')
lwp_model.fit(train_embeddings, targets)

C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\782853816.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(train_path)


In [41]:
for i in range(2, 11):
    process_dataset_with_knn(part_one_embed_dir, i, lwp_model, k=5, top_k=50)

Processing dataset 2 with kNN...


C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\3176197651.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(embed_path)


Dataset 2: Assigned pseudo-labels using kNN
Dataset 2: Updated prototypes for classes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Processing dataset 3 with kNN...


C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\3176197651.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(embed_path)


Dataset 3: Assigned pseudo-labels using kNN
Dataset 3: Updated prototypes for classes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Processing dataset 4 with kNN...


...

Dataset 8: Assigned pseudo-labels using kNN
Dataset 8: Updated prototypes for classes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Processing dataset 9 with kNN...


C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\3176197651.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(embed_path)


Dataset 9: Assigned pseudo-labels using kNN
Dataset 9: Updated prototypes for classes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Processing dataset 10 with kNN...


C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\3176197651.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load(embed_path)


Dataset 10: Assigned pseudo-labels using kNN
Dataset 10: Updated prototypes for classes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [ ]:
for i in range(11, 21):
    process_dataset_with_knn(part_two_embed_dir, i, lwp_model, k=5, top_k=50)

In [43]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import os
import torch

# Function to evaluate on eval embeddings
def evaluate_on_eval_embeddings(embed_dir, dataset_idx, model, ground_truth=None):
    """
    Evaluate the trained model using eval embeddings.
    Args:
        embed_dir (str): Path to the directory containing eval embeddings.
        dataset_idx (int): Dataset index (e.g., 1 for D1, 2 for D2, etc.).
        model (LWP or kNN): Trained model for predictions.
        ground_truth (np.array): True labels for eval set (if available).
    Returns:
        dict: Evaluation metrics (if ground_truth provided).
    """
    # Load eval embeddings
    embed_path = os.path.join(embed_dir, f'eval_embeds_{dataset_idx}.pt')
    eval_embeddings = torch.load(embed_path)

    # Predict labels using the trained model
    print(f"Evaluating on dataset {dataset_idx}...")
    predicted_labels = model.predict(eval_embeddings)

    if ground_truth is not None:
        # Compute evaluation metrics if ground truth is available
        accuracy = accuracy_score(ground_truth, predicted_labels)
        report = classification_report(ground_truth, predicted_labels, zero_division=0)
        print(f"Accuracy on eval set {dataset_idx}: {accuracy * 100:.2f}%")
        print("Classification Report:")
        print(report)
        return {"accuracy": accuracy, "report": report}
    else:
        # No ground truth available
        print(f"Predictions on eval set {dataset_idx}: {predicted_labels[:10]}...")
        return {"predicted_labels": predicted_labels}

# Directories for eval embeddings
part_one_eval_dir = 'part_1_vit_embeds'
part_two_eval_dir = 'part_2_vit_embeds'

# Evaluate on D1 (with ground truth)
eval_labels_path = os.path.join('dataset', 'part_one_dataset', 'eval_data', '1_eval_data.tar.pth')
eval_data = torch.load(eval_labels_path)
eval_ground_truth = eval_data['targets']

# Evaluate on D1 using LWP model
evaluate_on_eval_embeddings(part_one_eval_dir, dataset_idx=1, model=lwp_model, ground_truth=eval_ground_truth)

# Evaluate on D2-D10 (unlabeled datasets)
for i in range(2, 11):
    evaluate_on_eval_embeddings(part_one_eval_dir, dataset_idx=i, model=lwp_model)



C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\917197337.py:45: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval_data = torch.load(eval_labels_path)
C:\Users\AN

Evaluating on dataset 1...
Accuracy on eval set 1: 86.56%
Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.88      0.90       252
           1       0.89      0.95      0.92       217
           2       0.99      0.65      0.79       264
           3       0.71      0.90      0.79       242
           4       0.77      0.89      0.83       257
           5       0.93      0.71      0.81       252
           6       0.95      0.92      0.93       269
           7       0.77      0.90      0.83       233
           8       0.95      0.93      0.94       266
           9       0.89      0.95      0.92       248

    accuracy                           0.87      2500
   macro avg       0.88      0.87      0.86      2500
weighted avg       0.88      0.87      0.86      2500

Evaluating on dataset 2...


C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\917197337.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval_embeddings = torch.load(embed_path)


Predictions on eval set 2: [3 5 0 1 0 0 7 8 8 0]...
Evaluating on dataset 3...


C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\917197337.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval_embeddings = torch.load(embed_path)


...

Predictions on eval set 8: [3 5 4 2 0 1 1 4 5 9]...
Evaluating on dataset 9...


C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\917197337.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval_embeddings = torch.load(embed_path)


Predictions on eval set 9: [7 3 8 5 3 7 8 1 3 6]...
Evaluating on dataset 10...


C:\Users\ANUSHKA SINGH\AppData\Local\Temp\ipykernel_27144\917197337.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  eval_embeddings = torch.load(embed_path)


Predictions on eval set 10: [2 9 3 4 9 5 8 3 4 5]...


In [ ]:
# Evaluate on D11-D20 (unlabeled datasets with different distribution)
for i in range(11, 21):
    evaluate_on_eval_embeddings(part_two_eval_dir, dataset_idx=i, model=lwp_model)

In [ ]:

# # Train LWP model on unlabeled 

# for i in range(2,11):
#     train_path=os.path.join(train_dir, f'{i}_train_data.tar.pth')
#     print(f"Processing dataset {i} from {train_path}")

#     dataset=torch.load(train_path, weights_only=False)
#     data= dataset['data']
#     data=data.reshape(-1,3072)
#     print(f"Reshaped dataset shape: {data.shape}")  # Should be (N, 3072)

#     data = normalize(data.reshape(data.shape[0],-1))  # Normalize data (important for distance calculations)
#     # Prepare DataLoader for the dataset
#     tensor_dataset = torch.tensor(data, dtype=torch.float32)
#     dataloader = DataLoader(data, batch_size=32, shuffle=False)
#     confidence_scores = []
#     embeddings = []
#     predictions = []
#     for batch in dataloader:
#         print(f"Batch shape: {batch.shape}")  # Should be (batch_size, 3072)

#     with torch.no_grad():
#         for batch in dataloader:

#             inputs = batch.numpy()
#             print(inputs.shape)
#             batch_predictions = []
#             batch_distances = []
#             print("Prototypes:", lwp_model.prototypes)

#             # Predict pseudo-labels and calculate distances to prototypes
#             for sample in inputs:
#                 print(f"Sample shape before flattening: {sample.shape}")
#                 sample = sample.flatten()  # Ensure the sample is a 1D vector
#                 print(f"Sample shape after flattening: {sample.shape}")
#                 dist_to_prototypes = {label: lwp_model.distance_fn(sample, proto) for label, proto in lwp_model.prototypes.items()}
#                 closest_label = min(dist_to_prototypes, key=dist_to_prototypes.get)
#                 closest_distance = dist_to_prototypes[closest_label]
#                 batch_predictions.append(closest_label)
#                 batch_distances.append(closest_distance)

#             predictions.extend(batch_predictions)
#             confidence = 1 / (1 + np.array(batch_distances))  # Convert distances to confidence scores
#             confidence_scores.extend(confidence.tolist())
#             embeddings.extend(inputs)

#     embeddings = np.array(embeddings)

#     # Step 4: Select top 50% most confident samples
#     sorted_indices = np.argsort(confidence_scores)[::-1]  # Sort by confidence scores (descending)
#     top_50_percent_indices = sorted_indices[:len(sorted_indices) // 2]

#     top_50_embeddings = embeddings[top_50_percent_indices]
#     top_50_predictions = np.array(predictions)[top_50_percent_indices]

#     # Step 5: Construct class centroids
#     class_centroids = {}
#     for label in np.unique(top_50_predictions):
#         class_embeddings = top_50_embeddings[top_50_predictions == label]
#         if class_embeddings.size > 0:
#             centroid = class_embeddings.mean(axis=0)
#             class_centroids[label] = centroid

#     # Print centroids
#     print(f"Class centroids calculated for dataset {i}:")
#     for label, centroid in class_centroids.items():
#         print(f"Class {label}: Centroid = {centroid[:5]}...")
   
